In [1]:
import os
import json
from typing import List
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from bs4 import BeautifulSoup

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
PATH_TO_CHROME_DRIVER = 'B:\\Users\\ekfon\\chromeDriver\\chromedriver.exe'

MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [3]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        """
        begin Selenium equivalent of requests:
        response = requests.get(url, headers=headers)
        self.body = response.content #which is then passed on to bs
        """
        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service(PATH_TO_CHROME_DRIVER)
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        page_source = driver.page_source
        driver.quit()
        #end Selenium part
        
        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage title: \"{self.title}\"\nWebpage contents:\n{self.text}\n\n"

In [4]:
#system prompt for the link anthology
anthology_sPrompt = "I'll provide you with a list of links from a webpage. \
You are able to decide which links are most relevant to include in a brochure about the company, \
such as the About page, any Company page, or a jobs/careers page.\n"

anthology_sPrompt += "You will respond in JSON format, providing full https URLs, just like in this example:\n"

anthology_sPrompt += """
{
    "links": [
        {"type": "about page", "url": "https://www.example-url.com/about"}
        {"type": "careers page", "url": "https://further.example-url.co.uk/Careers/"}
    ]
}
"""

In [5]:
def get_anthology_user_prompt(website):
    user_prompt = f"Below is the list of links from the webpage {website.url}. "
    user_prompt += "Please decide which of the links are relevant for a brochure about the company. \
Respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links."
    user_prompt += "Here is the list of links (some might be relative links):\n\n"
    user_prompt += "\n".join(website.links)

    return user_prompt

In [6]:
def get_links_anthology(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": anthology_sPrompt},
            {"role": "user", "content": get_anthology_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result) #because result is a string, and what we want is an actual dictionary

In [7]:
def get_all_details(url):
    result = "Landing page:\n\n"
    result += Website(url).get_contents()
    links = get_links_anthology(url)

    for link in links["links"]: #remember that links is a json dictionary
        result += f"\n\n{link['type']}\n"
        result += Website(link['url']).get_contents()

    return result

In [ ]:
print(get_all_details("https://edwarddonner.com"))

In [8]:
brochure_sPrompt = "You analyze the content of several relevant pages from a company's website. \
You use that knowledge to create a short brochure about the company. Your brochure is for prospective customers, investors, and recruits. \
Include details of company culture, customers, and job openings if you have the information. Respond in Markdown."

In [9]:
def get_brochure_user_prompt(company_name, url):
    prompt = f"You are looking at the website of the company called {company_name}.\n"
    prompt += "Here are the contents of its landing page and other relevant pages. Based on this content, \
create a short brochure of the company in Markdown:\n\n"
    prompt += get_all_details(url)
    prompt = prompt[:5_000] #this limits the prompt input, just in case
    
    return prompt

In [10]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
        {"role": "system", "content": brochure_sPrompt},
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    return result

In [11]:
translation_sPrompt = "You are a powerful translation tool. You will be given a brochure in Markdown format. \
Translate the brochure to French. Maintain the Markdown formatting, and output the translation in Markdown. \
Output only the clean, ready-to-use translation without any further comments."

In [12]:
def get_translation_user_prompt(company_name, url):
    prompt = f"Here is the {company_name} brochure for you. Translate it to French:\n\n"
    prompt += create_brochure(company_name, url)

    return prompt

In [15]:
def create_French_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
        {"role": "system", "content": translation_sPrompt},
        {"role": "user", "content": get_translation_user_prompt(company_name, url)}
        ]
    )
    results = response.choices[0].message.content
    display(Markdown(results))   
    

In [ ]:
answer = create_brochure("Anthropic", "https://www.anthropic.com/")

In [13]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
        {"role": "system", "content": brochure_sPrompt},
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
reponsee = create_French_brochure("Edward Donner", "https://edwarddonner.com")

```markdown
# Bienvenue chez Edward Donner

Chez **Edward Donner**, nous sommes à la pointe de l'utilisation de l'intelligence artificielle pour révolutionner le paysage du recrutement. Notre approche novatrice combine une IA générative de pointe avec un modèle de correspondance propriétaire pour créer une expérience fluide pour les recruteurs à la recherche de talents de premier plan.

## Notre Mission
Guidé par le concept japonais d'**Ikigai**, notre objectif ultime est de donner aux individus les moyens de découvrir leur potentiel et de poursuivre leur passion dans le domaine professionnel. Nous croyons qu'en alignant les bonnes personnes avec les bons rôles, nous pouvons élever la prospérité humaine—ce qui est nécessaire dans un monde où 77 % des employés déclarent ne pas se sentir inspirés au travail.

## Culture d'Entreprise
Notre culture est ancrée dans un mélange de créativité, d'innovation technologique et d'ardeur à construire une communauté. Nous encourageons les dialogues ouverts et les collaborations, où chaque membre de l'équipe a la possibilité de contribuer et de grandir. Nous sommes fiers de notre esprit entrepreneurial, qui se manifeste chez notre fondateur, Ed Donner, qui partage une passion pour le codage, la musique et l'apprentissage continu.

## Nos Clients
Nous avons établi des partenariats fructueux avec une variété d'entreprises via notre plateforme, en nous concentrant sur le secteur du recrutement pour améliorer les processus de sourcing et d'engagement. Nos clients bénéficient de correspondances plus rapides et plus précises sans se fier uniquement aux mots-clés, garantissant qu'ils découvrent les meilleurs candidats pour leurs équipes.

## Rejoignez Notre Équipe
Nous sommes à la recherche de penseurs innovants qui sont enthousiasmés par l'IA et son impact sur l'avenir du travail. Si vous recherchez un lieu de travail dynamique où vos contributions font la différence et où vous pouvez vous épanouir, nous vous invitons à consulter nos dernières offres d'emploi sur notre [page carrière](#).

## Connectez-vous Avec Nous !
Êtes-vous tout aussi enthousiaste à propos de l'IA et du recrutement que nous le sommes ? Connectons-nous ! Nous valorisons le réseautage et la collaboration, que ce soit autour d'un café virtuel ou lors d'une rencontre en personne à NYC, nous serions ravis d'avoir de vos nouvelles.

**Informations de Contact :**
- **Site Web :** [www.edwarddonner.com](http://www.edwarddonner.com)
- **Email :** ed [at] edwarddonner [dot] com
- **Suivez-nous :** [LinkedIn](#) | [Twitter](#) | [Facebook](#)

Rejoignez-nous dans notre quête pour changer la façon dont les gens se connectent à leurs carrières !
```